In [118]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import spacy
import random
from kerastuner.tuners import RandomSearch
import kerastuner as kt
import tensorflow as tf
from keras.utils import np_utils
import preprocess as pr
import seaborn as sns
import pickle

In [69]:
dfc = pd.read_csv("data/intents.csv").sample(frac=1, random_state=42).reset_index(drop=True)
df2 = dfc[["text","intent"]]

pd.isna(df2['text']).sum()

1

In [74]:
df2 = df2.fillna("")

In [75]:
pd.isna(df2['text']).sum()

0

In [76]:
classes = dfc.intent.unique().tolist()
df2['intent'] = df2['intent'].apply(classes.index)

In [77]:
df2['intent']

0       0
1       1
2       2
3       3
4       4
       ..
205    22
206     8
207    16
208    30
209    22
Name: intent, Length: 210, dtype: int64

In [78]:
val_label_cor = pd.concat(
    {
        "intent_val" : df2['intent'],
        "intent_label" : dfc['intent']
    },
    axis = 1
)

val_label_cor = val_label_cor.drop_duplicates().reset_index(drop=True)
val_label_cor

,intent_val,intent_label
0,0,thanks
1,1,type1_treatment
2,2,define_gestational_diabetes
3,3,locate_endocrino
4,4,type1_symptoms
5,5,diabetes_risk_factors
6,6,define_diabetes
7,7,change_lifestyle
8,8,greeting
9,9,hypoglycemia_symptoms


In [79]:
X = np.stack(df2["text"].apply(pr.wordvec).to_numpy(), axis=0)
y = df2.intent.to_numpy()

X_train = X[:int(0.7*X.shape[0])]
y_train = y[:int(0.7*y.shape[0])]
X_test = X[int(0.7*X.shape[0]):]
y_test = y[int(0.7*y.shape[0]):]

In [80]:
#preparer les labels
y_train = y_train.tolist()
y_test = y_test.tolist()

def to_vec(val):
    vec = [0] * len(classes)
    vec[val] = 1
    return vec

y_train = np.array(list(map(to_vec, y_train)))
y_test = np.array(list(map(to_vec, y_test)))

print(y_test.shape, '\n', y_test)

(63, 33) 
 [[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]]


In [81]:
#Definissons un modele sequentiel Keras de 3 couches
test_model = Sequential()
test_model.add(Dense(300, input_dim=300, activation='relu')) #dim des vecteurs vaut 300
test_model.add(Dense(150, activation='relu'))
test_model.add(Dense(y_train[0].size, activation='sigmoid'))
test_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Le gradient descent pour optimiser
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
test_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = test_model.fit(X_train, y_train, epochs=200, validation_split=0.2, batch_size=5, verbose=1)

 loss: 0.1076 - accuracy: 0.9316 - val_loss: 1.3822 - val_accuracy: 0.7333
Epoch 62/200
24/24 [==============================] - 0s 7ms/step - loss: 0.1150 - accuracy: 0.9487 - val_loss: 1.3678 - val_accuracy: 0.7000
Epoch 63/200
24/24 [==============================] - 0s 7ms/step - loss: 0.1473 - accuracy: 0.9402 - val_loss: 1.3221 - val_accuracy: 0.7667
Epoch 64/200
24/24 [==============================] - 0s 9ms/step - loss: 0.0989 - accuracy: 0.9402 - val_loss: 1.3667 - val_accuracy: 0.6667
Epoch 65/200
24/24 [==============================] - 0s 9ms/step - loss: 0.1036 - accuracy: 0.9402 - val_loss: 1.3190 - val_accuracy: 0.7000
Epoch 66/200
24/24 [==============================] - 0s 8ms/step - loss: 0.0985 - accuracy: 0.9316 - val_loss: 1.4023 - val_accuracy: 0.7000
Epoch 67/200
24/24 [==============================] - 0s 8ms/step - loss: 0.1036 - accuracy: 0.9573 - val_loss: 1.4241 - val_accuracy: 0.7333
Epoch 68/200
24/24 [==============================] - 0s 7ms/step - loss:

In [82]:
_, accuracy = test_model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

2/2 [==============================] - 0s 28ms/step - loss: 2.3582 - accuracy: 0.6190
Accuracy: 61.90


## Hyperparameter tuning

In [83]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(X_train.shape[1],)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=300, max_value=1000, step=10)
  activ = hp.Choice('activation', values = ['relu', 'sigmoid', 'softmax', 'tanh'])
  model.add(Dense(units=hp_units, activation=activ))
  
  #2nd layer
  hp_units2 = hp.Int('units2', min_value=30, max_value = hp_units, step = 10)
  activ2 = hp.Choice('activation2', values = ['relu', 'sigmoid', 'softmax', 'tanh'])
  model.add(Dense(units=hp_units2, activation=activ2))
  #output layer 
  model.add(Dense(len(classes), activation='sigmoid'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [84]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=50,
                     factor=3,
                     directory='keras-model',
                     project_name='hyperpara_tuning')

In [85]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [86]:
tuner.search(X_train, y_train, epochs=200, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 90 Complete [00h 00m 13s]
val_accuracy: 0.800000011920929

Best val_accuracy So Far: 0.800000011920929
Total elapsed time: 00h 07m 31s
INFO:tensorflow:Oracle triggered exit


In [87]:
print(f"""meilleur units 1 : {best_hps.get('units')}\nmeilleur activation 1 : {best_hps.get('activation')}\nmeilleur units 2 : {best_hps.get('units2')}\nmeilleur activation 1 : {best_hps.get('activation2')}\nmeilleur learning rate : {best_hps.get('learning_rate')}.
""")

meilleur units 1 : 440
meilleur activation 1 : tanh
meilleur units 2 : 90
meilleur activation 1 : tanh
meilleur learning rate : 0.001.



In [88]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=200, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Meilleure epoch: %d' % (best_epoch,))

och 60/200
4/4 [==============================] - 0s 33ms/step - loss: 0.1213 - accuracy: 0.9487 - val_loss: 1.2522 - val_accuracy: 0.7333
Epoch 61/200
4/4 [==============================] - 0s 62ms/step - loss: 0.1186 - accuracy: 0.9573 - val_loss: 1.2602 - val_accuracy: 0.7333
Epoch 62/200
4/4 [==============================] - 0s 26ms/step - loss: 0.1159 - accuracy: 0.9487 - val_loss: 1.2470 - val_accuracy: 0.7333
Epoch 63/200
4/4 [==============================] - 0s 32ms/step - loss: 0.1129 - accuracy: 0.9573 - val_loss: 1.2402 - val_accuracy: 0.7333
Epoch 64/200
4/4 [==============================] - 0s 35ms/step - loss: 0.1098 - accuracy: 0.9573 - val_loss: 1.2386 - val_accuracy: 0.7333
Epoch 65/200
4/4 [==============================] - 0s 66ms/step - loss: 0.1096 - accuracy: 0.9573 - val_loss: 1.2412 - val_accuracy: 0.7333
Epoch 66/200
4/4 [==============================] - 0s 36ms/step - loss: 0.1064 - accuracy: 0.9573 - val_loss: 1.2355 - val_accuracy: 0.7333
Epoch 67/200
4/

In [111]:
"""
checkpoint_path = "training_keras_hypermodel/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
"""
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/36
/home/rahima/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/backend.py:4869: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(
4/4 [==============================] - 1s 100ms/step - loss: 3.4865 - accuracy: 0.0855 - val_loss: 3.2511 - val_accuracy: 0.2667

Epoch 00001: saving model to training_keras_hypermodel/cp.ckpt
Epoch 2/36
4/4 [==============================] - 0s 16ms/step - loss: 2.9206 - accuracy: 0.3333 - val_loss: 3.0745 - val_accuracy: 0.2333

Epoch 00002: saving model to training_keras_hypermodel/cp.ckpt
Epoch 3/36
4/4 [==============================] - 0s 21ms/step - loss: 2.5627 - accuracy: 0.4017 - val_loss: 3.0011 - val_accuracy: 0.2333

Epoch 00003: saving model to training_keras_hypermodel/cp.ckpt
Epoch 4/36
4/4 [==============================] - 0s 19ms/step - loss: 2.3016 - a

In [112]:
eval_result = hypermodel.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

2/2 [==============================] - 0s 5ms/step - loss: 1.4920 - accuracy: 0.5714
[test loss, test accuracy]: [1.491957426071167, 0.5714285969734192]


In [113]:
hypermodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 440)               132440    
_________________________________________________________________
dense_1 (Dense)              (None, 90)                39690     
_________________________________________________________________
dense_2 (Dense)              (None, 33)                3003      
Total params: 175,133
Trainable params: 175,133
Non-trainable params: 0
_________________________________________________________________


## Prédire
<br/>
Les sorties sont des probabilites

In [114]:
predictions = hypermodel.predict(X_test)
predictions

array([[0.2670649 , 0.0717662 , 0.20869943, ..., 0.39284045, 0.42900172,
        0.16672283],
       [0.22466317, 0.9612142 , 0.54913586, ..., 0.82641375, 0.38262457,
        0.32160655],
       [0.21008238, 0.40869057, 0.1488249 , ..., 0.3857499 , 0.36714917,
        0.30379176],
       ...,
       [0.16375619, 0.3431636 , 0.99659586, ..., 0.7385855 , 0.1466837 ,
        0.27672613],
       [0.36131233, 0.9968251 , 0.8556698 , ..., 0.94515413, 0.18988153,
        0.23535505],
       [0.14269277, 0.2518988 , 0.9954884 , ..., 0.67510533, 0.1142939 ,
        0.37323713]], dtype=float32)

In [115]:
val_label_cor.iloc[np.argmax(predictions[0])]['intent_label']

'gestational_diabetes_risk_factors'

In [116]:
text = "what's diabetes"
dd = np.array([pr.wordvec(text)])
pp = hypermodel.predict(dd)

In [117]:
val_label_cor.iloc[np.argmax(pp[0])]['intent_label']

'define_diabetes'

## Enregistrer les modèles pour les réutiliser 
<br/>

On enregesitre la totalite du modele

In [104]:
test_model.save("saved_models_vars/bot_test_model.h5")
hypermodel.save("saved_models_vars/bot_hypermodel.h5")

In [119]:
pickle.dump(val_label_cor, open('saved_models_vars/val_label_cor.pckl', 'wb'))